<a href="https://colab.research.google.com/github/hbisgin/BigDatav1/blob/main/Lecture5_InClass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Example").getOrCreate()
sc = spark.sparkContext

In [ ]:
dataRDD = sc.parallelize([
  ("Brooke", 20), ("Denny", 31),
  ("Jules", 30), ("TD", 35), ("Brooke", 25)
])

# dataRDD is assumed to contain pairs like: (key, value)

agesRDD = (
    dataRDD
      # 1) For each (key, value), initialize a (sum, count) pair as (value, 1)
      #    Example: ("alice", 34) -> ("alice", (34, 1))
      .map(lambda x: (x[0], (x[1], 1)))

      # 2) Reduce by key to add up sums and counts across the same key
      #    ("alice", (34, 1)) and ("alice", (30, 1))
      #    -> ("alice", (34+30, 1+1)) = ("alice", (64, 2))
      .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))

      # 3) For each key, convert (sum, count) -> average = sum / count
      #    ("alice", (64, 2)) -> ("alice", 32.0)
      #    NOTE: cast to float if you need non-integer division in older Python.
      .map(lambda x: (x[0], x[1][0] / x[1][1]))
)


agesRDD.collect()

[('Brooke', 22.5), ('Denny', 31.0), ('Jules', 30.0), ('TD', 35.0)]

In [ ]:
from pyspark.sql.functions import avg
# if you want to import all available functions and call them as needed, from pyspark.sql import functions as F. Then you can call like F.avg

data_df = spark.createDataFrame([
  ("Brooke", 20), ("Denny", 31),
  ("Jules", 30), ("TD", 35), ("Brooke", 25)
], ["name", "age"])

avg_df = data_df.groupBy("name").agg(avg("age"))
avg_df.show()

+------+--------+
|  name|avg(age)|
+------+--------+
|Brooke|    22.5|
| Denny|    31.0|
| Jules|    30.0|
|    TD|    35.0|
+------+--------+



How to define schema

In [ ]:
from pyspark.sql.types import *
schema = StructType([StructField("author", StringType(), False),
  StructField("title", StringType(), False),
  StructField("pages", IntegerType(), False)])

schema.fieldNames()
data = [("Chris Bishop", "Deep Learning", 346)]
df = spark.createDataFrame(data, schema=schema)
df.show()
print(df.printSchema())


+------------+-------------+-----+
|      author|        title|pages|
+------------+-------------+-----+
|Chris Bishop|Deep Learning|  346|
+------------+-------------+-----+

root
 |-- author: string (nullable = false)
 |-- title: string (nullable = false)
 |-- pages: integer (nullable = false)

None


In [ ]:
df.schema

StructType([StructField('author', StringType(), False), StructField('title', StringType(), False), StructField('pages', IntegerType(), False)])

If you want to use DDL

In [ ]:
schemaDDL = "author STRING, title STRING, pages INT"
df_ddl = spark.createDataFrame(data, schema=schemaDDL)
df_ddl.show()
print(df_ddl.printSchema())

+------------+-------------+-----+
|      author|        title|pages|
+------------+-------------+-----+
|Chris Bishop|Deep Learning|  346|
+------------+-------------+-----+

root
 |-- author: string (nullable = true)
 |-- title: string (nullable = true)
 |-- pages: integer (nullable = true)

None


In [ ]:
df_ddl.schema

StructType([StructField('author', StringType(), True), StructField('title', StringType(), True), StructField('pages', IntegerType(), True)])

Columns & Expressions

In [ ]:
titanic = spark.read.csv("/content/drive/MyDrive/DATA/titanic.csv", header=True, inferSchema=True)
titanic.show()

+---+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|_c0|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+---+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|  0|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|  1|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|  2|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|  3|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|  4|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
|  5|          6|       0|     3|    Moran, Mr. 

In [ ]:
from pyspark.sql.functions import *
print(titanic.columns)
titanic.select(expr("Fare * 2")).show(2)
titanic.select(col("Fare") * 2).show(3)

['_c0', 'PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked']
+----------+
|(Fare * 2)|
+----------+
|      14.5|
|  142.5666|
+----------+
only showing top 2 rows

+----------+
|(Fare * 2)|
+----------+
|      14.5|
|  142.5666|
|     15.85|
+----------+
only showing top 3 rows



Average fare for each class

In [ ]:
titanic.groupBy("Pclass").agg(avg("Fare")).show()

+------+------------------+
|Pclass|         avg(Fare)|
+------+------------------+
|     3|13.525700639658854|
|     1| 84.74390547263675|
|     ?|  36.0694693877551|
|     2| 20.63643430232558|
+------+------------------+



In [ ]:
titanic.withColumn(
    "passenger_info",
      concat(
        lit("Class "), col("Pclass"), lit(" - "), col("Name")
    )
).show(5)


+---+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+--------------------+
|_c0|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|      passenger_info|
+---+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+--------------------+
|  0|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|Class 3 - Braund,...|
|  1|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|Class 1 - Cumings...|
|  2|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|Class 3 - Heikkin...|
|  3|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|Class 1 - Futrell...|
|  4|          5|   

In [ ]:
titanic.withColumn("Expensive", expr("Fare > 50")).show(5)

+---+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+---------+
|_c0|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|Expensive|
+---+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+---------+
|  0|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|    false|
|  1|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|     true|
|  2|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|    false|
|  3|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|     true|
|  4|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|  

Row objects

In [ ]:
from pyspark.sql import Row
blog_row = Row(6, "Reynold", "Xin", "https://tinyurl.6", 255568, "3/2/2015",
  ["twitter", "LinkedIn"])
# access using index for individual items, or slicing for multiple items
blog_row[1:3]

('Reynold', 'Xin')

You can create dataframes by using Row objects

In [ ]:
rows = [Row("Matei Zaharia", "CA"), Row("Reynold Xin", "CA")]
authors_df = spark.createDataFrame(rows, ["Authors", "State"])
authors_df.show()

+-------------+-----+
|      Authors|State|
+-------------+-----+
|Matei Zaharia|   CA|
|  Reynold Xin|   CA|
+-------------+-----+



Using DataFrame Reader and DataFrameWriter

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

titanic_schema = StructType([
    StructField("PassengerId", IntegerType(), True),
    StructField("Survived", IntegerType(), True),
    StructField("Pclass", IntegerType(), True),
    StructField("Name", StringType(), True),
    StructField("Sex", StringType(), True),
    StructField("Age", DoubleType(), True),       # age can be fractional / null
    StructField("SibSp", IntegerType(), True),    # siblings/spouses aboard
    StructField("Parch", IntegerType(), True),    # parents/children aboard
    StructField("Ticket", StringType(), True),
    StructField("Fare", DoubleType(), True),
    StructField("Cabin", StringType(), True),
    StructField("Embarked", StringType(), True)   # port of embarkation
])


In [ ]:
titanic_df = spark.read \
    .option("header", True) \
    .schema(titanic_schema) \
    .csv("/content/drive/MyDrive/DATA/titanic.csv")

titanic_df_alternative = spark.read.csv("/content/drive/MyDrive/DATA/titanic.csv", header=True, schema=titanic_schema)

titanic_df.printSchema()
titanic_df.show(5, truncate=False)

titanic_df_alternative.printSchema()
titanic_df_alternative.show(5, truncate=False)

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)

+-----------+--------+------+----+---------------------------------------------------+----+-----+-----+------+--------+-------+--------+
|PassengerId|Survived|Pclass|Name|Sex                                                |Age |SibSp|Parch|Ticket|Fare    |Cabin  |Embarked|
+-----------+--------+------+----+---------------------------------------------------+----+-----+-----+------+--------+-------+--------+
|0          |1       |0     |3   |Braund, Mr. Owen Harris                            |NULL|NULL |1    |0     |NULL    |7.25   

How to write?

In [ ]:
titanic_df.write.saveAsTable("titanic_table")

In [ ]:
df2 = spark.sql("SELECT * FROM titanic_table")
df2.show()

+-----------+--------+------+----+--------------------+----+-----+-----+------+--------+-------+--------+
|PassengerId|Survived|Pclass|Name|                 Sex| Age|SibSp|Parch|Ticket|    Fare|  Cabin|Embarked|
+-----------+--------+------+----+--------------------+----+-----+-----+------+--------+-------+--------+
|          0|       1|     0|   3|Braund, Mr. Owen ...|NULL| NULL|    1|     0|    NULL|   7.25|    NULL|
|          1|       2|     1|   1|Cumings, Mrs. Joh...|NULL| NULL|    1|     0|    NULL|71.2833|     C85|
|          2|       3|     1|   3|Heikkinen, Miss. ...|NULL| NULL|    0|     0|    NULL|  7.925|    NULL|
|          3|       4|     1|   1|Futrelle, Mrs. Ja...|NULL| NULL|    1|     0|113803.0|   53.1|    C123|
|          4|       5|     0|   3|Allen, Mr. Willia...|NULL| NULL|    0|     0|373450.0|   8.05|    NULL|
|          5|       6|     0|   3|    Moran, Mr. James|NULL| NULL|    0|     0|330877.0| 8.4583|    NULL|
|          6|       7|     0|   1|McCarthy, Mr

Exercise: Can you read the Blue Jays dataset and report the average Skull size for each gender (KnownSex)?

In [ ]:
import pandas as pd
bj = pd.read_csv("/content/drive/MyDrive/DATA/Blue_Jays.csv") #this code is for you to view the data
bj.head()

,Unnamed: 0,X,BirdID,KnownSex,BillDepth,BillWidth,BillLength,Head,Mass,Skull,Sex
0,1,1,0000-00000,M,8.26,9.21,25.92,56.58,73.30,30.66,1
1,2,2,1142-05901,M,8.54,8.76,24.99,56.36,75.10,31.38,1
2,3,3,1142-05905,M,8.39,8.78,26.07,57.32,70.25,31.25,1
3,4,4,1142-05907,F,7.78,9.30,23.48,53.77,65.50,30.29,0
4,5,5,1142-05909,M,8.71,9.84,25.47,57.32,74.90,31.85,1
